#### Dependencies

In [1]:
import pandas as pd

# Import SQLAlchemy 'automap' and other dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy import MetaData, create_engine, inspect, Column, Integer, String, Numeric, Text, Float, Date
import sqlite3

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

#### Read in cleaned (CSV) data

In [2]:
file_path_measurements = "Resources/clean_hawaii_measurements.csv"
file_path_stations = "Resources/clean_hawaii_stations.csv"

clean_data_measurements = pd.read_csv(file_path_measurements)
clean_data_stations = pd.read_csv(file_path_stations)

In [3]:
#Give the first column in each table the header 'id'
clean_data_measurements = clean_data_measurements.rename(columns={'Unnamed: 0': 'id'})
clean_data_stations = clean_data_stations.rename(columns={'Unnamed: 0': 'id'})

In [4]:
clean_data_measurements.head()

,id,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,5,USC00519397,2010-01-07,0.06,70


In [5]:
clean_data_measurements.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18103 entries, 0 to 18102
Data columns (total 5 columns):
id         18103 non-null int64
station    18103 non-null object
date       18103 non-null object
prcp       18103 non-null float64
tobs       18103 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 707.2+ KB


In [6]:
clean_data_stations.head(10)

,id,station,name,latitude,longitude,elevation
0,0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


#### Create the database for later use

In [7]:
# Define our measurements table
class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.measurement}"
    
# Define our stations table
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.station}"

In [8]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

#Create the database
Base.metadata.create_all(engine)


In [9]:
#Check that table names are correct
engine.table_names()

['measurement', 'station']

In [10]:
#Import csv data into sqlite database
clean_data_measurements.to_sql('measurement', con=engine, if_exists='append', index=False)
clean_data_stations.to_sql('station', con=engine, if_exists='append', index=False)

In [11]:
#Check that data was imported correctly
engine.execute("SELECT * FROM measurement LIMIT 5").fetchall()

[(0, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (1, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (2, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (3, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]

In [12]:
engine.execute("SELECT * FROM station LIMIT 5").fetchall()

[(0, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (1, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (2, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (3, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (4, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]